<a href="https://colab.research.google.com/github/mauriciodev/spatialgeodesy/blob/main/exercises/Ex6_gnss_positioning_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 6 - Basic GNSS positioning
In this exercise we are going to donwload the observation and navigation files from a RBMC station and compute the position on the first epoch using the **GPS L1 pseudorange**. After that, the student will bring

# Preparation

In [ ]:
#Python standard libraries
import os #File path operations.
import shutil #Shell operations. Unzipping, moving files, etc.
import urllib.request #Downloader.

#External libraries
import numpy as np #Numeric Python.
import pandas as pd #Python Data Analysis Library.
import matplotlib.pyplot as plt #Plots.
import xarray as xa #Multi dimension arrays. For georinex.

from scipy.constants import c #lightspeed


In [ ]:
# Installing the package that reads rinex and sp3 files
!pip install git+https://github.com/geospace-code/georinex
#!pip install georinex
#brute force fix for a bug in keplerian.py
!sed -i -e 's/sv\["sv"\] in {\"R\", \"S\"}/sv\["sv"\] in ("R", "S")/g' /usr/local/lib/python3.10/dist-packages/georinex/keplerian.py
import georinex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/geospace-code/georinex to /tmp/pip-req-build-c9tnocnx
  Running command git clone --filter=blob:none --quiet https://github.com/geospace-code/georinex /tmp/pip-req-build-c9tnocnx
  Resolved https://github.com/geospace-code/georinex to commit c689a5a6bc2ffb68bc055f150f1da1b6bab12812
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
#this method uses numpydatetime64 instead of pandas timestamp because of georinex.
def secondsOfGPSWeek(t):
    t=t.astype('datetime64[ns]')
    day=t.astype('datetime64[D]')
    secsOfDay=(t-day)/ np.timedelta64(1, 's')
    gps_t0=np.datetime64('1980-01-06T00:00:00')
    dayOfWeek=((t - gps_t0)/ np.timedelta64(1, 'D') % 7).astype(np.int64)
    return secsOfDay+dayOfWeek*24*60*60

#distance between two 3-dim vectors
def dist(v1,v2):
    return np.linalg.norm(v1[:3]-v2[:3]) #ignoring everything after the 3rd dimension

## Obtaining data

In [ ]:
#This is the reference time. We are going to use it to download the files.
t = pd.to_datetime('2023-01-01 02:00:01')
station="RJNI" #This is the station.

In [ ]:
url_v2=f"https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/{t.year}/{t.day_of_year:03}/{station.lower()}{t.day_of_year:03}1.zip"
print("URL for the version 2 RINEX file:", url_v2)

#splitting the file name from the URL
obsRinexFile_v2=os.path.split(url_v2)[1]
urllib.request.urlretrieve(url_v2, obsRinexFile_v2) #download the file saving as the name in obsRinexFile_v2
print(obsRinexFile_v2)

# the ! prefix runs the commands on a linux shell. So we can use both python and shell commands on this notebook.
shutil.unpack_archive(obsRinexFile_v2)
uncompressed=obsRinexFile_v2[:-3]

for f in os.listdir(): print(f) #listing files in the current directory

URL for the version 2 RINEX file: https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/2023/001/rjni0011.zip
rjni0011.zip
.config
rjni0011.23d
rjni0011.zip
rjni0011.23g
rjni0011.23n
rjni0011.23l
sample_data


In [ ]:
#reading the obs and nav files
obsData=georinex.load(uncompressed+f'{t.year%100}d', use='G')
navData=georinex.load(uncompressed+f'{t.year%100}n')

# First, lets get $t_{rec}$:

In [ ]:
#getting the first epoch of the observation file. We could change this to other epochs.
epoch=obsData.time[0].values
epoch

numpy.datetime64('2023-01-01T00:00:00.000000000')

# Combining observation and navigation datas on each SV
Here we use XArray merge method to join observations and navigation data. We select the epoch, remove null values and combine the datasets. Please observe that the same dataset now has the information to compute the SV position and the observations obtained by the receiver using the signals from that SV.

In [ ]:
#selecting only SVs with data on the current epoch.
epochObs=obsData.sel(time=epoch).dropna(dim='sv', how='all')
#Get the nav data of the current epoch. We're using "pad" to get the latest.
#Note that the orbits are valid for longer periods (hours), so we still have to compute
#the positions with the time difference dt.
epochOrbits=navData.sel(time=epoch, method="pad")
#Drop any sv without data
epochOrbits=epochOrbits.dropna(dim='sv')
#merging navigation and observation data using dim SV
#We used right join so that only sats with observations remain
epochData=xa.merge([epochOrbits, epochObs],join='right')
del epochObs
del epochOrbits
epochData

<xarray.Dataset>
Dimensions:           (sv: 9)
Coordinates:
  * sv                (sv) <U3 'G01' 'G03' 'G04' 'G06' ... 'G16' 'G26' 'G30'
    time              datetime64[ns] 2023-01-01
Data variables: (12/46)
    SVclockBias       (sv) float64 0.0002302 -0.0003743 ... 0.0002395 -0.0005343
    SVclockDrift      (sv) float64 -5.002e-12 1.137e-12 ... 6.821e-13 7.958e-13
    SVclockDriftRate  (sv) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
    IODE              (sv) float64 79.0 74.0 160.0 73.0 31.0 39.0 59.0 99.0 56.0
    Crs               (sv) float64 -65.25 -13.25 -19.84 ... 94.25 86.53 9.312
    DeltaN            (sv) float64 3.858e-09 4.19e-09 ... 5.093e-09 5.038e-09
    ...                ...
    P2                (sv) float64 2.347e+07 2.058e+07 ... 2.412e+07 nan
    S1                (sv) float64 38.9 49.6 47.5 41.1 47.8 46.0 46.4 39.6 38.3
    S2                (sv) float64 21.8 40.3 34.8 22.9 35.5 34.9 29.6 33.5 nan
    S5                (sv) float64 44.0 54.7 50.8 45.0 nan 49.3 nan 43.6 41.7
    S7                (sv) float64 nan nan nan nan nan nan nan nan nan
    S8                (sv) float64 nan nan nan nan nan nan nan nan nan
Attributes:
    version:    2.11
    svtype:     ['G']
    rinextype:  nav
    filename:   rjni0011.23n

# Find the clock bias

In [ ]:
# Time of ephemeris is the same for all GPS orbits in this epoch, so we get the first.
toe=epochData.Toe.values[0]
toe #in GPS seconds of the week

0.0

In [ ]:
t_epoch=secondsOfGPSWeek(epoch)-toe
t_epoch

0.0

In [ ]:
#Satellite clock bias.
#We are assigning it to the "dt_sat" variable on the epochData xarray dataset.
epochData=epochData.assign(dt_sat=epochData.SVclockBias+ epochData.SVclockDrift*t_epoch + epochData.SVclockDriftRate* (t_epoch**2))
epochData.dt_sat

<xarray.DataArray 'dt_sat' (sv: 9)>
array([ 2.30218749e-04, -3.74278519e-04, -5.77154569e-05,  5.50809782e-04,
        2.66006216e-04, -2.44179275e-04, -5.25041483e-04,  2.39532441e-04,
       -5.34310471e-04])
Coordinates:
  * sv       (sv) <U3 'G01' 'G03' 'G04' 'G06' 'G07' 'G09' 'G16' 'G26' 'G30'
    time     datetime64[ns] 2023-01-01

# Computing the SV coordinates of the current epoch.
The epoch is registered by the receiver, so the time that the satellite (SV) emitted that signal ($t_{sat}$) can be computed with the pseudorange
\begin{gather}
PR=c.\Delta T = c.(t_{sat}-t_{rec}) \\
t_{sat} = t_{rec}-\Delta T
\end{gather}
The signal took $\Delta T$ seconds to reach the receiver, so the SV position should be computed at $t_{rec}-\Delta T$. But we also have a clock bias on each satellite.

In [ ]:
#pseudorange/lightspeed is the time taken to reach the receiver.
#We are assinging it to the "Delta_t_trans" variable on the epochData.
epochData=epochData.assign(Delta_t_trans=epochData.C1/c+epochData.dt_sat )
epochData

<xarray.Dataset>
Dimensions:           (sv: 9)
Coordinates:
  * sv                (sv) <U3 'G01' 'G03' 'G04' 'G06' ... 'G16' 'G26' 'G30'
    time              datetime64[ns] 2023-01-01
Data variables: (12/48)
    SVclockBias       (sv) float64 0.0002302 -0.0003743 ... 0.0002395 -0.0005343
    SVclockDrift      (sv) float64 -5.002e-12 1.137e-12 ... 6.821e-13 7.958e-13
    SVclockDriftRate  (sv) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
    IODE              (sv) float64 79.0 74.0 160.0 73.0 31.0 39.0 59.0 99.0 56.0
    Crs               (sv) float64 -65.25 -13.25 -19.84 ... 94.25 86.53 9.312
    DeltaN            (sv) float64 3.858e-09 4.19e-09 ... 5.093e-09 5.038e-09
    ...                ...
    S2                (sv) float64 21.8 40.3 34.8 22.9 35.5 34.9 29.6 33.5 nan
    S5                (sv) float64 44.0 54.7 50.8 45.0 nan 49.3 nan 43.6 41.7
    S7                (sv) float64 nan nan nan nan nan nan nan nan nan
    S8                (sv) float64 nan nan nan nan nan nan nan nan nan
    dt_sat            (sv) float64 0.0002302 -0.0003743 ... 0.0002395 -0.0005343
    Delta_t_trans     (sv) float64 0.07853 0.06828 0.07156 ... 0.08069 0.08219
Attributes:
    version:    2.11
    svtype:     ['G']
    rinextype:  nav
    filename:   rjni0011.23n

In [ ]:
satPos=[]
obs=[]
for sv in epochData.sv:
  #We are changing the "time" dimension so that the orbit is calculated on a different time.

  sv_orbit = epochData.sel(sv=sv)
  delta_t=np.timedelta64(int(sv_orbit.Delta_t_trans.values.item()*1e9),'ns')
  sv_orbit['time']=[epoch-delta_t]
  x,y,z=georinex.keplerian2ecef(sv_orbit)
  satPos.append([*x,*y,*z,sv_orbit.dt_sat.values])
  obs.append(sv_orbit.C1)

satPos=np.array(satPos).squeeze()
print("X,Y,Z, dt")
print(satPos)
obs=np.array(obs)
print(obs)


X,Y,Z, dt
[[ 1.32943313e+07 -1.68507553e+07  1.50984622e+07  2.30218749e-04]
 [ 2.24235897e+07 -1.27287513e+07 -6.25404279e+06 -3.74278519e-04]
 [ 1.31477681e+07 -8.36477516e+06 -2.15169719e+07 -5.77154569e-05]
 [-6.83952767e+06 -2.02230261e+07 -1.56752732e+07  5.50809782e-04]
 [ 6.08065640e+06 -2.54433427e+07 -2.19193950e+06  2.66006216e-04]
 [ 1.94441289e+06 -1.69348957e+07 -2.04359424e+07 -2.44179275e-04]
 [ 2.44237096e+07  1.40800566e+06 -1.11957078e+07 -5.25041483e-04]
 [ 1.64253051e+07  8.34563086e+06 -1.93472619e+07  2.39532441e-04]
 [-2.29685406e+06 -2.47208010e+07  9.15959108e+06 -5.34310471e-04]]
[23474275.25  20582672.086 21470707.148 23517948.656 21421807.828
 22326329.063 22763024.758 24117063.094 24800349.805]


## Least squares fitting (theory)
\begin{gather}
L_b=AX+V
\end{gather}
$L_b$ : observations vector} \
$A$ Design matrix \\
$X$ Estimated parameters vector} \
\begin{gather}
\text{Least Squares fitting minimizes }V= L_b- AX \\
\text{So we want the minumum } \Phi \\
\Phi = V^tPV = (L_b- AX)^T P (L_b-AX) \\
P=\sigma_0^2 \Sigma_{L_B}^1 \\ \\
\text{The solution is} \\
\hat X =(A^T P A)^{-1}A^T P L_b \ \text{(estimated parameters)} \\
\Sigma_{\hat X}=\hat \sigma_0^2 (A^T P A)^{-1} \ \text{(variance covariance matrix)} \\
\hat \sigma_0^2= \frac{V^TPV}{n-u} \ \text{(variance a posteriori)} \\
\end{gather}
$n$: the number of observations \\
$u$: the number of parameters \\
$\Sigma_{L_B}$ observations variance covariance matrix \\
$\sigma_0^2$ variance a priori.


## Design matrix
GNSS design matrixes are usually nonlinear because we measure the distance we want to find $X_{rec}$.
$\rho=\|X_{rec}- X_{sat}\|= \sqrt{ (x_{rec}- x_{sat})^2 + (y_{rec}- y_{sat})^2 + (z_{rec}- z_{sat})^2}$ \
Thus we linearize the model using the first component of a Taylor series and iteratively compute the parameters vector $X_{i+1} = X_i + \Delta X$
\begin{gather}
A_L \Delta X = E\{\Delta L\}
\end{gather}

Complete pseudorange equation:
\begin{gather}
R_{P_f}= \rho + c(dt_{rec}-dt^{sat}) + Tr +\alpha_f \cdot STEC + K_{P_f,rec} - K_{P_f}^{sat} + M_{P_f} + \epsilon_{P_f}$
\end{gather}
Pseudorange $R_{P_f}$, geometric distance $\rho$, of a signal $P$ on the frequency $f$ and wavelength $\lambda_{P_f}$, light speed $c$, $dt_{rec}$ and $dt^{sat}$ are the clock delays (receiver and satellite), $Tr$ is the tropospheric delay, $\alpha_f \cdot STEC$ is the ionospheric effect, proportional to the total electron content. $K_{P_f,rec}$ and $K_{P_f}^{sat}$ are the hardware biases. $M_{P_f}$ is the multipath error.

The single point positioning cannot start with the full equation without prior knowledge. So we use a simplified version of the same equation. \\
\begin{gather}
R_{P_f}= \rho + c(dt_{rec}-dt^{sat}) \\
R_{P_f}= \sqrt{ (x_{rec}- x_{sat})^2 + (y_{rec}- y_{sat})^2 + (z_{rec}- z_{sat})^2} + c(dt_{rec}-dt^{sat})
\end{gather}
The receiver measures $R_{P_f}$ and we want to compute the receiver position. The GNSS orbits let us compute the satellite position and the clock bias of each satellite ($dt^{sat}$). So the receiver clock bias ($dt_{rec}$) should be estimated on the adjustment, along with $x_{rec}, y_{rec},z_{rec}$.

A=\begin{bmatrix}
\frac{x_{rec_{i-1}}- x_{sat_1}}{\rho_{sat_1}} & \frac{y_{rec_{i-1}}- y_{sat_1}}{\rho_{sat_1}} & \frac{z_{rec_{i-1}}- z_{sat_1}}{\rho_{sat_1}} & 1\\
... \\
\frac{x_{rec_{i-1}}- x_{sat_n}}{\rho_{sat_n}} & \frac{y_{rec_{i-1}}- y_{sat_n}}{\rho_{sat_n}} & \frac{z_{rec_{i-1}}- z_{sat_n}}{\rho_{sat_n}} & 1
\end{bmatrix}
$\Delta X$=\begin{bmatrix} x_{rec_i}-x_{rec_{i-1}} \\ y_{rec_i}-y_{rec_{i-1}} \\ z_{rec_i}-z_{rec_{i-1}} \\ c \cdot dt_{rec_i} - c \cdot dt_{rec_{i-1}} \end{bmatrix}
$\Delta L_b$=\begin{bmatrix}
R_{p_{f}:sat_1} -\rho_{sat_1} +c \cdot dt_{sat_1} \\
R_{p_{f}:sat_2} -\rho_{sat_2} +c \cdot dt_{sat_2} \\
 ... \\
R_{p_{f}:sat_1} -\rho_{sat_n} +c \cdot dt_{sat_n}
\end{bmatrix}


# Now you are going to implement the formulaes above for A and $\Delta L_b$

In [ ]:
#FILL THIS PART OF THE CODE
#Please create a method that creates returns the jacobian A matrix of this epoch
#Inputs:
#  X0: position of the receiver on current iteration (we need to refine it).
#  satPos: the matrix with the positions of each SV compute with the orbits.

X0=np.array([0,0,0,0]) #x,y,z and c.dt
def makeJacobian(x0, xsat):
  #FILL THIS PART OF THE CODE

  #first, compute the number of satellites on xsat

  #compute the difference between the current receiver coordinates X0 and each sat

  #compute the distance between X0 and each sat and store all of them on a vector called rho0

  #Create the matrix A using the formula above. Note that the clock column is always 1.



  return A, rho0 #the method should return both the jacobian and the current euclidean distances.

#Test if it's working
A, rho0 = makeJacobian(X0[:3],satPos[:,:3])
A, rho0

(array([[-0.50660209,  0.64212541, -0.57535144,  1.        ],
        [-0.84514993,  0.47974938,  0.23571622,  1.        ],
        [-0.4948882 ,  0.31485409,  0.80990899,  1.        ],
        [ 0.25823992,  0.76356043,  0.59185101,  1.        ],
        [-0.23163058,  0.96921382,  0.0834976 ,  1.        ],
        [-0.07306523,  0.63636285,  0.76792174,  1.        ],
        [-0.90779786, -0.05233376,  0.41613005,  1.        ],
        [-0.61480633, -0.3123806 ,  0.72417645,  1.        ],
        [ 0.08679484,  0.93416387, -0.3461279 ,  1.        ]]),
 array([26242156.14874261, 26532084.96256958, 26567147.90545819,
        26485167.7496153 , 26251526.95903601, 26612011.84562018,
        26904348.03040737, 26716226.27994587, 26463024.14765548]))

In [ ]:
#FILL THIS PART OF THE CODE
#Please build the observations vector L of this epoch
#Inputs:
#  rho0: vector of euclidean distances.
#  sat_clock_correction: vector of clock biases.
#  observations: vector of pseudorange measures.

def makeL(rho0, sat_clock_correction, observations):
    #FILL THIS PART OF THE CODE


#test the method
L = makeL(rho0, satPos[:,3], obs)
L

array([-2698863.0539543 , -6061618.75384791, -5113743.41615666,
       -2802090.47517955, -4749972.47357613, -4358885.88763121,
       -4298726.74925074, -2527353.16671748, -1822856.5921716 ])

In [ ]:
xyzSat=satPos
#We began with X at the center of the Earth. Note that an out of earth solution may exist, but we should not converge to that.
X0=np.array([0,0,0,0])

nsat=len(xyzSat)
print("Number of observations: ", nsat)
print("Degrees of freedom: ", nsat-4)
print("X0:", X0)
#cloning so I can perform Xsat-X0
cdt_r=0
i=0
dX0X1=1 #initialing X1 <> X0
X1=X0
Lb=obs
#compute 20 epochs or until the difference between the estimated coordinates is smaller than 0.01m
while dX0X1 > 0.01 and (i<20):
    i+=1
    X0=X1

    #Using the makeJacobian method implemeneted previously.
    A, rho0 = makeJacobian(X0[:3], satPos[:,:3])


    #We could compute satellite elevation, tropospheric and ionospheric delays here.
    #The makeL method would require some changes.

    #Using the makeL method.
    L=makeL(rho0, xyzSat[:,3], Lb )

    #weights are the identity at first, then we use the variance covariance matrix from previous iteration.
    if i==1:
        P = np.identity(nsat, int) #* 0.8*np.sin(elev[:,0])
    else:
        P=MVC_L

    #least squares
    AtPA_inv=np.linalg.inv(A.T @ P @ A)
    X_fit=AtPA_inv @ A.T @ P @ L

    #updating the coordinates with the Delta X computed with least squares.
    X1=X0+X_fit

    #computing residuals and MVC matrixes
    V=L-A @ X_fit
    sigma2=V.T @ P @ V / (nsat-4)
    MVC_X=sigma2*AtPA_inv
    MVC_L=A @ MVC_X @ A.T

    #computing the distance between the coordinates on this epoch and previous.
    dX0X1=dist(X1,X0)
    print(dX0X1)
    #print("Clock bias: {}".format(cdt_r/c))

print("Estimated X,Y,Z,dt_rec",X1)


Number of observations:  9
Degrees of freedom:  5
X0: [0 0 0 0]
7474175.723394334
1076139.7190455126
23495.332312530478
11.696376761056738
0.05559231015630259
4.315706761620355e-05
Estimated X,Y,Z,dt_rec [ 4289674.21339282 -4018706.12229054 -2467098.26558084  1223039.7560608 ]


In [ ]:
print("Coordinates provided by RBMC:", obsData.position)
print("Distance from the coordinates computed to the coordinate provided by RMBC:", dist(obsData.position, X1), "(m)")

Coordinates provided by RBMC: [4289663.4011, -4018945.7498, -2467135.8382]
Distance from the coordinates computed to the coordinate provided by RMBC: 242.7961092660253 (m)
